In [2]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.offline import iplot
import skimage.io as sio
from skimage.transform import resize
from PIL import Image

import json
import pickle

In [5]:
with open('../Result/tmp/tmp1.pickle', 'rb') as file:
    result_sum1, result1, df1 = pickle.load(file)

with open('../Result/tmp/tmp2.pickle', 'rb') as file:
    result_sum2, result2, df2 = pickle.load(file)

In [ ]:
def visualize_3d_pose(pose_3d, visual_json,image_path_list):
    scale = 2.3
    file_path = visual_json
    
    with open(file_path, "r") as file:
        json_data = json.load(file)
    
    landmarks_3d = pose_3d

    # 각 슬라이드에 해당하는 데이터를 저장할 리스트를 초기화
    frames_data = []
    # 각 프레임마다 처리합니다.
    for i in range(len(landmarks_3d)):
        frame_landmarks = landmarks_3d[i]  # 현재 프레임의 랜드마크
    
        # 각 랜드마크를 점으로 표현하는 Scatter3d 객체를 생성합니다.
        scatter3d = go.Scatter3d(
            x=frame_landmarks[:, 0]*scale-1.2,
            y=frame_landmarks[:, 1]*scale-1.2,
            z=frame_landmarks[:, 2]*scale,
            mode='markers',
            marker=dict(
                size=2,
                color='blue',  # 점의 색상을 파란색으로 지정합니다. 필요에 따라 변경 가능합니다.
                opacity=0.8
            ),
            showlegend=False
            #name=f'frame{i}'
        )
    
        # 라인을 그릴 랜드마크 쌍을 이용하여 선을 그립니다.
        line_frame_data = []
        for landmark_pair in json_data.keys():
            for j in json_data[landmark_pair]:
                x_values = [frame_landmarks[j, 0]*scale-1.2, frame_landmarks[int(landmark_pair), 0]*scale-1.2]
                y_values = [frame_landmarks[j, 1]*scale-1.2, frame_landmarks[int(landmark_pair), 1]*scale-1.2]
                z_values = [frame_landmarks[j, 2]*scale, frame_landmarks[int(landmark_pair), 2]*scale]
    
                line = go.Scatter3d(
                    x=x_values,
                    y=y_values,
                    z=z_values,
                    mode='lines',
                    line=dict(color='red', width=3),  # 선의 색상과 두께를 지정합니다. 필요에 따라 변경 가능합니다.
                    showlegend=False
                    #name=f'frame_line_{i}_{int(landmark_pair)}_{j}'
                )
    
                line_frame_data.append(line)  # 현재 프레임의 데이터에 선을 추가합니다.
    
        # 현재 프레임의 데이터를 Frame 객체로 생성하고 frames_data 리스트에 추가합니다.
        image = sio.imread(f"example_image_frame_list_0/{image_path_list[i]}")
        image = np.array(Image.fromarray(image).resize((128,128)))
        
        x = np.linspace(-1, 1, 128)
        y = np.linspace(-0.9, 0.9, 128)
        x, y = np.meshgrid(x, y)
        z = np.sin(x**2*y)
        
        #image = sio.imread("https://raw.githubusercontent.com/empet/Discrete-Arnold-map/master/Images/cat-128.jpg")
        img = image[:, :, 1] 
        z = -1 * np.ones(x.shape)  # 이미지를 표시할 y 값 생성
        
        img_fig = go.Surface(x=x, y=-y, z=z,  # 이미지를 y=0.5 평면에 표시
                             surfacecolor=np.fliplr(np.flipud(img)),  # 이미지 데이터
                             colorscale='gray', 
                             showscale=False)
        frame = go.Frame(data=[scatter3d, *line_frame_data,img_fig], name=f'frame{i}')
        frames_data.append(frame)
    
    steps = []
    for i in range(len(landmarks_3d)):
        step = dict(
            method="animate",
            args=[
                [f"frame{i}"],
                dict(
                    frame=dict(duration=500, redraw=True),
                    fromcurrent=True,
                    transition=dict(duration=300, easing="quadratic-in-out"),
                ),
            ],
            label=f"{i}"
        )
        steps.append(step)
    
    sliders = [dict(
        active=0,
        steps=steps,
        pad={"t": 50}
    )]
    
    # 레이아웃을 설정합니다.
    layout = go.Layout(
        title='3D Pose Visualization',
        scene=dict(
            xaxis=dict(title='',  # x 축 레이블 없애기
            #tickvals=[],  # x 축 눈금 값 없애기
            #ticktext=[],  # x 축 눈금 텍스트 없애기
            range=[-1,1]
            ),
            yaxis=dict(title='',  # y 축 레이블 없애기
            #tickvals=[],  # y 축 눈금 값 없애기
            #ticktext=[],  # y 축 눈금 텍스트 없애기
            range=[-1,1]
            ),
            zaxis=dict(title='',  # z 축 레이블 없애기
            #tickvals=[],  # z 축 눈금 값 없애기
            #ticktext=[],  # z 축 눈금 텍스트 없애기
            range=[-2,2]                       
            ),
            dragmode='orbit'  # 회전 및 확대/축소만 허용하는 모드
        ),
        width=600,
        height=400,
        sliders=sliders,
        # updatemenus=[dict(
        #     type="buttons",
        #     buttons=[dict(label="Play",
        #   #                method="animate",
        #                   args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)]
        #                  )]
        # )]
    )
    
    # Figure 객체를 생성합니다.
    fig = go.Figure(data=[scatter3d, *line_frame_data, img_fig], layout=layout, frames=frames_data)
    fig.update_layout(width=1000, height=1000, 
                  scene_camera_eye_x=0,
                  scene_camera_eye_y=0,
                  scene_camera_eye_z=1.5, 
                  scene_aspectratio=dict(x=0, y=1, z=1))
    iplot(fig)

In [ ]:
#시각화 실행 코드
visual_json = "visualize_pose.json"
#for i in range(len(video_list)):
for i in range(1):
    pose_json = f"json/output_poses_{i}.json"
    pose_3d = convert_json2Array(pose_json)
    image_path_list = sorted(os.listdir(f"example_image_frame_list_{i}"))[1:]
    visualize_3d_pose(pose_3d, visual_json,image_path_list)